In [20]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [21]:
df = sns.load_dataset("mpg")

X_train, y_train = df[~df["horsepower"].isna()][["displacement", "acceleration"]], df[~df["horsepower"].isna()]["horsepower"]
X_pred = df[df["horsepower"].isna()][["displacement", "acceleration"]]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_pred)
y_pred = np.round(y_pred)
df.loc[X_pred.index, "horsepower"] = y_pred
df = pd.get_dummies(df.drop("name", axis = 1), columns = ["origin"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1


In [22]:
X, y = df.drop(["mpg"], axis = 1).values, df["mpg"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [23]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [344]:
sigmoid = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))

def seed_np(integer):
    np.random.seed(integer)

class ERegressor:
    class Net:
        def __init__(self):
            pass

        def set_layers(self, layers):
            self.layers = layers
        
        def set_weights(self, weights_and_biases):
            self.w = []

            for weight_bias in weights_and_biases:
                self.w += [weight_bias]

        
        def set_mutation_sigma(self, n):
            self.mutation_sigma = 3 - (3 - 1200 / (200 + n))
        
        def predict(self, input):
            forward_pass = input.T

            for i in range(0, len(self.layers) - 2):
                forward_pass = sigmoid(self.w[i].T @ forward_pass)

            forward_pass = self.w[-1].T @ forward_pass

            return forward_pass.reshape(-1)

        
        def get_layer_shapes(self):
            layer_shapes = []

            for w in self.w:
                layer_shapes += [w.shape]

            return layer_shapes
        
        def __add__(self, other):
            w = []

            for i in range(len(self.layers) - 1):
                w += [(self.w[i] + other.w[i]) / 2 + np.random.normal(0, self.mutation_sigma, (self.layers[i], self.layers[i + 1]))]

            return w

    
    def __init__(self, n = 100, hidden_layers = False, random_state = None, verbose = 0):
        self.nets = {}
        self.y_preds = {}
        self.best_net = None
        self.best_result = None
        self.n = n // 2 * 2
        if not hidden_layers:
            self.layers = [1]
        else:
            self.layers = hidden_layers + [1]
        
        for i in range(self.n):
            self.nets[i] = self.Net()

        if random_state != None:
            seed_np(random_state)

        self.verbose = verbose
    
    def fit(self, X_train, y_train, epochs = 100):
        
        X_train = np.c_[np.ones(X_train.shape[0]), X_train]
        

        self.layers = [X_train.shape[1]] + self.layers

        for key in self.nets.keys():
            self.nets[key].set_layers(self.layers)
            
            w = []

            for i in range(len(self.layers) - 1):
                w += [np.random.uniform(-3, 3, (self.layers[i], self.layers[i + 1]))]
            
            self.nets[key].set_weights(w)

        for epoch in range(epochs):

            if self.verbose == 1:
                print(f"Epoch {epoch}")

            for key, _ in self.nets.items():
                self.y_preds[key] = self.nets[key].predict(X_train)
                self.nets[key].set_mutation_sigma(epoch)
            
            self.mean_absolute_errors = {}
            
            for key, _ in self.y_preds.items():
                self.mean_absolute_errors[key] = mean_absolute_error(y_train, self.y_preds[key])

            self.sorted_indecies = [key for key, _ in sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])]

            if self.best_result != None:
                if self.best_result[1] > sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0][1]:
                    self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                    print(f"Epoch {epoch}: MAE: {self.best_result[1]}")
            else:        
                self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                print(f"Epoch {epoch}: MAE: {self.best_result[1]}")

            self.best_net = self.sorted_indecies[0]
            
            for i in range(0, self.n // 2, 2):
                self.nets[self.sorted_indecies[self.n // 2 + i]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
                self.nets[self.sorted_indecies[self.n // 2 + i + 1]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
                pass

    def predict(self, X):
        X = np.c_[np.ones(X.shape[0]), X]
        
        return self.nets[self.best_net].predict(X)

In [343]:
regressor = ERegressor(n = 100, hidden_layers = [10, 10], random_state = 42, verbose = 0)
regressor.fit(scaled_X_train, y_train, epochs = 100)

Epoch 0: MAE: 15.296019642054345
Epoch 1: MAE: 6.252617393070794
Epoch 3: MAE: 5.228662066310539
Epoch 8: MAE: 5.2131485391661965
Epoch 17: MAE: 4.660412245124438
Epoch 27: MAE: 4.621293525125449
Epoch 43: MAE: 4.450837251864408
Epoch 55: MAE: 4.430673213859081
Epoch 64: MAE: 4.284189659784247
Epoch 65: MAE: 4.223830949639475
Epoch 79: MAE: 4.114065518552848
Epoch 87: MAE: 4.036498459841673
Epoch 93: MAE: 4.008075511993223


In [314]:
regressor.nets[regressor.best_net].get_layer_shapes()

[(10, 6), (6, 1)]

In [284]:
regressor.nets[0].w[0]

array([[  1.88486375,   4.09054216, -12.70383698,   1.60136995,
         -0.10647768,   1.52567257,  -3.66154699,   2.38752505,
         17.21335415,   1.52890912],
       [ -1.04662901,   3.91601317,  -3.40888948,   3.62819865,
          1.49499236,   2.36173467,  -4.86867134,   8.03994509,
          0.4381468 ,  -4.78293194],
       [ -4.81143831,  -6.24138865,   8.49448522,  -6.37166523,
         -9.50777365,   3.11112169,  -3.79093769,  -2.25542966,
         -1.55364881,   3.90307181],
       [  2.06700965,   1.27743179,  -2.87605543, -14.53725201,
         -4.01749327,  -4.70097437,  -3.76528493,  -4.21299475,
          6.25647505,  -0.16425083],
       [ -3.07810538,  -8.88877333,  -9.83495269,   1.1761453 ,
         -7.28651892,   8.85586792, -10.95816544,  -7.3346799 ,
         -4.77182957, -10.07564909],
       [  1.48154477,   8.82171785,   1.28948789,   2.37877693,
         -6.99354729,   1.23525921,  -3.08585689,  -0.81289428,
         -0.07826975,   3.14988382],
       [ 1

In [174]:
scaled_X_train.T.shape

(9, 318)

In [285]:
regressor.nets[1].get_layer_shapes()

[(10, 10), (10, 1)]

In [318]:
y_pred = regressor.predict(scaled_X_test)

In [319]:
mean_absolute_error(y_test, y_pred)

1.912881422101075